In [2]:
# Economic 

In [40]:
import numpy as np
# from scipy.optimize import minimize

In [41]:
def calculate_total_cost(welfare_program, population):
    total_cost = 0

    # Loop over the joint distribution
    for i in range(100):  # loop over wealth percentiles
        for j in range(100):  # loop over income percentiles
            average_income = population.income_percentiles[j]
            average_wealth = population.wealth_percentiles[i]

            # Compute benefit for an individual in this bin
            E = .5  # Eligibility set to constant .5 for now - we can change to a function of wealth and income percentiles
            individual_benefit = E * min(welfare_program.G + welfare_program.S * average_income, welfare_program.M, 
                                         max([welfare_program.M - welfare_program.T * (average_income - welfare_program.P), 0]))

            # Multiply the individual benefit by number of individuals in this bin to get total benefit for the bin
            total_benefit_for_bin = individual_benefit * population.number_of_individuals(i, j)

            # Calculate total cost of providing the benefit for this bin
            bin_cost = total_benefit_for_bin / welfare_program.efficiency

            # Accumulate the cost
            total_cost += bin_cost

    return total_cost

In [42]:
import numpy as np

# Classes for Individual, Firm, and Government
class Individual:
    def __init__(self, income, wealth=0):
        self.income = income
        self.wealth = wealth
        self.tax_paid = 0

class Firm:
    def __init__(self, net_income, wealth):
        self.net_income = net_income
        self.wealth = wealth
        self.tax_paid = 0

class Population:
    def __init__(self, total_count, wealth_percentiles, income_percentiles, joint_distribution):
        self.total_count = total_count
        self.wealth_percentiles = wealth_percentiles  # Array of length 100 for wealth values
        self.income_percentiles = income_percentiles  # Array of length 100 for income values
        self.joint_distribution = joint_distribution  # 100x100 array with joint distribution of wealth and income

    def number_of_individuals(self, wealth_index, income_index):
        """Returns the number of individuals for a specific wealth and income percentile."""
        percentage = self.joint_distribution[wealth_index][income_index]
        return self.total_count * percentage

class Government:
    def __init__(self):
        self.revenue = 0
        self.non_welfare_expenditure = 1000000  # Placeholder
        # Other government budget attricutes

    def total_expenditure(self, welfare_programs, population):
        welfare_expenditures = sum([calculate_total_cost(program, population) for program in welfare_programs])
        return self.non_welfare_expenditure + welfare_expenditures


class WelfareProgram:
    def __init__(self, G, S, M, P, T, participation_rate=1.0, efficiency=1.0):
        self.G = G  # Grant for an individual with 0 income
        self.S = S  # Subsidy on earned income
        self.M = M  # Maximum allowable grant
        self.P = P  # Phase-out income
        self.T = T  # "Tax rate" on income above phase-out
        self.participation_rate = participation_rate  # Participation rate
        self.efficiency = efficiency  # Rate of rebates given per dollar of funding

    def calculate_rebate(self, income):
        return min(self.G + self.S * income, self.M, max(self.M - self.T * (income - self.P), 0))



# Test with some example welfare programs
welfare_program1 = WelfareProgram(G=5000, S=0.1, M=10000, P=30000, T=0.05, efficiency=0.7, participation_rate=0.9)
welfare_program2 = WelfareProgram(G=3000, S=0.15, M=7000, P=25000, T=0.07, efficiency=0.8, participation_rate=1.0)
welfare_programs = [welfare_program1, welfare_program2]

In [43]:
# Global constant for poverty line
POVERTY_LINE = 12700  # Placeholder value, adjust as needed

# UBI policy definition
class UBIProgram(WelfareProgram):
    def __init__(self, alpha, efficiency=1.0):
        G = alpha * POVERTY_LINE
        S = 0  # No subsidy on earned income for UBI
        M = G  # Maximum allowable grant is the same as G for UBI
        P = float('inf')  # No phase-out for UBI
        T = 0  # No tax rate on income above phase-out for UBI
        participation_rate = 1.0  # 100% participation rate
        super().__init__(G, S, M, P, T, efficiency, participation_rate)

In [44]:
class ProgressiveTax:
    def __init__(self, brackets):
        """
        Initialize with a dictionary representing the tax brackets.

        :param brackets: Dictionary where `income: rate` represents the marginal tax rate beginning at `income` and
                         ending at the value of the next key.
        """
        # Sort the brackets based on income
        self.brackets = dict(sorted(brackets.items()))

    def compute_tax(self, income):
        """
        Compute the tax for a given income based on the progressive tax brackets.

        :param income: Income for which to compute the tax.
        :return: Computed tax.
        """
        tax = 0
        previous_bracket_limit = 0

        # Iterate through the sorted brackets
        for bracket_limit, rate in self.brackets.items():
            # If income is within the current bracket
            if income <= bracket_limit:
                tax += (income - previous_bracket_limit) * rate
                return tax
            else:
                # Compute tax for the entire bracket range and move to the next
                tax += (bracket_limit - previous_bracket_limit) * rate
                previous_bracket_limit = bracket_limit

        # If income is beyond the highest bracket, apply the highest rate to the remainder
        tax += (income - previous_bracket_limit) * rate
        return tax

"""
# Example: uncomment to run

tax_brackets = {
    10000: 0.1,   # 10% for income <= 10,000
    50000: 0.2,  # 20% for income between 10,001 and 50,000
    100000: 0.3  # 30% for income > 50,000
}
progressive_tax = ProgressiveTax(tax_brackets)
print(progressive_tax.compute_tax(20000))  # Example usage
"""

'\n# Example: uncomment to run\n\ntax_brackets = {\n    10000: 0.1,   # 10% for income <= 10,000\n    50000: 0.2,  # 20% for income between 10,001 and 50,000\n    100000: 0.3  # 30% for income > 50,000\n}\nprogressive_tax = ProgressiveTax(tax_brackets)\nprint(progressive_tax.compute_tax(20000))  # Example usage\n'

In [45]:
class TaxPolicy:
    def __init__(self, individual_tax_policy, corporate_tax_policy, other_tax_params):
        """
        Initialize the TaxPolicy with individual and corporate progressive tax policies.

        :param individual_tax_policy: A ProgressiveTax object for individual income tax.
        :param corporate_tax_policy: A ProgressiveTax object for corporate tax.
        :param other_tax_params: Parameters for other taxes. This can be expanded upon.
        """
        self.individual_tax_policy = individual_tax_policy
        self.corporate_tax_policy = corporate_tax_policy
        self.other_tax_params = other_tax_params

    def individual_income_tax(self, individual_income):
        """
        Compute individual income tax based on the given income and the tax policy.

        :param individual_income: The income for which to compute the tax.
        :return: Computed tax.
        """
        return self.individual_tax_policy.compute_tax(individual_income)

    def corporate_tax(self, corporate_income):
        """
        Compute corporate tax based on the given corporate income and the tax policy.

        :param corporate_income: The corporate income for which to compute the tax.
        :return: Computed tax.
        """
        return self.corporate_tax_policy.compute_tax(corporate_income)

    def other_taxes(self):
        # Compute other taxes.
        # Placeholder for now; this can be expanded upon based on the nature of other taxes.
        pass

    def total_individual_income_tax_revenue(self, population):
        """
        Compute the total revenue from individual income taxes based on the population's income distribution.

        :param population: Population object with joint distribution of income and wealth.
        :return: Total revenue from individual income taxes.
        """
        total_revenue = 0

        # Iterate over the income percentiles
        for j in range(100):
            average_income = population.income_percentiles[j]
            tax_for_income_bin = self.individual_income_tax(average_income)

            # For each wealth percentile, compute the number of individuals and add to the total revenue
            for i in range(100):
                total_revenue += tax_for_income_bin * population.number_of_individuals(i, j)

        return total_revenue

## Effects on individual behavior

We now switch to examining individual consumer behavior: namely, preferences detailing their consumption demand and labor supply functions.

Homogeneous Agents: We'll use a representative agent model, where one agent's behavior is representative of the entire population.

Focus on Consumption: Given an income (which might include UBI), the representative agent decides how much to consume. This will be based on a simple utility function of consumption.

Fixed Prices: Prices of goods and services are constant. We'll note to reintroduce inflationary considerations in future iterations.

Simple Tax and UBI System: We will implement the UBI policy and tax system we've defined so far to see how the introduction of UBI affects the representative agent's income and thus consumption.

Once we've modeled effects on individual behavior, we will aggregate these to the level of a Population of individuals with homogeneous preferences

Now, let's proceed by building the model with these considerations:

In [160]:
class Individual:
    def __init__(self, wage, elasticity_income, elasticity_wealth, total_hours=24, welfare_programs=[], tax_policy=None):
        self.wage = wage
        # Assume constant elasticity of income and wealth
        self.elasticity_income_income_effect = elasticity_income # ONLY MEASURING INCOME EFFECT Why? because wages DO NOT CHANGE, so there's no substitution effect
        # You can think about this as a bestowal of 1 good, but the price remains the same

        # TODO: We will also require an elasticity_income_substitution_effect when considering the effects of changing the tax policy to fund UBI
        
        self.elasticity_wealth_marshallian = elasticity_wealth # Compensated (Marshallian)
        self.total_hours = total_hours
        self.welfare_programs = welfare_programs
        self.tax_policy = tax_policy

    def non_labor_income(self, labor_hours):
        """Compute non-labor income based on welfare programs."""
        income = self.wage * labor_hours
        benefits = sum([program.calculate_rebate(income) for program in self.welfare_programs])
        return benefits

    def taxes_paid(self, labor_hours):
        """Compute taxes paid based on tax policy."""
        income = self.wage * labor_hours
        return self.tax_policy.compute_tax(income)

    # TODO: modify later to include substitution effect due to changing taxes. Note that we will also need a method to compute delta_wage_after_tax for 2 tax policies
    def adjust_labor_supply(self, delta_income, delta_wealth):
        """Adjust labor supply based on changes in income and wealth."""
        delta_l = self.elasticity_income_income_effect * delta_income + self.elasticity_wealth_marshallian * delta_wealth
        adjusted_hours = self.total_hours + delta_l * self.total_hours
        return max(0, min(self.total_hours, adjusted_hours))  # Ensuring labor supply is between 0 and total_hours

    def compute_net_income(self, labor_hours):
        """Compute net income after accounting for taxes and welfare benefits."""
        gross_income = self.wage * labor_hours
        total_benefits = self.non_labor_income(labor_hours)
        total_taxes = self.taxes_paid(labor_hours)
        return gross_income + total_benefits - total_taxes

Let's test our utility optimization. We'll consider an individual earning  wage of 25 under a progressive income tax poloicy and receiving benefits from an EITC program. 

In [161]:
tax_brackets = {
    9700: 0.10,
    39475: 0.12,
    84200: 0.22,
    160725: 0.24,
    204100: 0.32,
    510300: 0.35
}
tax_policy = ProgressiveTax(tax_brackets)

EITC_params = {
    'G': 0, 
    'S': 6557/19030,
    'M': 6557,
    'P': 19030,
    'T': (6557 / (50162 - 19030))
}
EITC_program = WelfareProgram(**EITC_params)


### Effects on labor supply for an individual (test)

In [162]:
wage = 25  # $/hour
initial_working_hours = 2000 # hours/year

individual = Individual(wage=wage, 
                        elasticity_income=-0.1, 
                        elasticity_wealth=-0.1, 
                        total_hours=initial_working_hours, 
                        welfare_programs=[EITC_program], 
                        tax_policy=tax_policy)

initial_net_income = individual.compute_net_income(initial_working_hours)
print(f"Initial net income: ${initial_net_income:.2f}")

delta_income = 12000 / initial_net_income
delta_wealth = 12000 / (initial_net_income + 100000) # Assume pre-income wealth of 100000

adjusted_hours = individual.adjust_labor_supply(delta_income, delta_wealth)
print(f"Adjusted labor supply: {adjusted_hours:.2f} hours/year")
# Our model predicts that someone working 2000 hours at $25 / hour pre-UBI with an EITC will work 1927.65 hours / year post-UBI - not taking into account the
# substitution effect of income changes

Initial net income: $43175.62
Adjusted labor supply: 1927.65 hours/year


### Effects on individual consumption